In [1]:

import mysql.connector  
import configparser
import json
from deepface import DeepFace
from deepface.commons import package_utils, folder_utils
import os

from datetime import datetime
import hashlib

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import math
import time

#!pip install deepface
from deepface import DeepFace
from deepface.commons import package_utils, folder_utils

In [2]:
model = DeepFace.build_model("Facenet")

In [3]:
facial_img_paths = []
#Available at: https://github.com/serengil/deepface/tree/master/tests/dataset
for root, directory, files in os.walk("../dataset"):
    for file in files:
        if '.jpg' in file:
            facial_img_paths.append(root+"/"+file)

In [4]:
instances = []
 
for i in tqdm(range(0, len(facial_img_paths))):
#for i in tqdm(range(0, 10)):
    facial_img_path = facial_img_paths[i]   
    embedding = DeepFace.represent(img_path = facial_img_path, model_name = "Facenet", enforce_detection=False)[0]["embedding"]
    objs = DeepFace.analyze(img_path = facial_img_path) 
    instance = []
    instance.append(facial_img_path)
    instance.append(embedding)
    instance.append(objs[0]["dominant_emotion"])
    instance.append(objs[0]["age"])
    instance.append(objs[0]["dominant_gender"])
    instance.append(objs[0]["dominant_race"])
    instances.append(instance)

100%|██████████| 57/57 [04:02<00:00,  4.25s/it]


In [5]:
df = pd.DataFrame(instances, columns = ["img_name", "embedding","dominant_emotion", "age", "dominant_gender", "dominant_race"])
df.head()

,img_name,embedding,dominant_emotion,age,dominant_gender,dominant_race
0,../dataset/couple.jpg,"[-0.9569708108901978, 0.10792163014411926, -1....",fear,31,Man,latino hispanic
1,../dataset/img1.jpg,"[0.8597482442855835, 0.9605178833007812, 1.345...",happy,31,Woman,latino hispanic
2,../dataset/img10.jpg,"[-0.3334352374076843, 1.2074142694473267, 1.05...",neutral,32,Woman,white
3,../dataset/img11.jpg,"[0.4563964009284973, 0.19060522317886353, 1.72...",fear,32,Woman,white
4,../dataset/img11_reflection.jpg,"[0.7846647500991821, 0.7575387954711914, 1.717...",neutral,36,Woman,white


In [41]:

# Assuming your DataFrame is named df
df['img_name'] = df['img_name'].str.replace('../dataset/', '')

# Display the modified DataFrame
df.head()


,img_name,embedding,dominant_emotion,age,dominant_gender,dominant_race
0,couple.jpg,"[-0.9569708108901978, 0.10792163014411926, -1....",fear,31,Man,latino hispanic
1,img1.jpg,"[0.8597482442855835, 0.9605178833007812, 1.345...",happy,31,Woman,latino hispanic
2,img10.jpg,"[-0.3334352374076843, 1.2074142694473267, 1.05...",neutral,32,Woman,white
3,img11.jpg,"[0.4563964009284973, 0.19060522317886353, 1.72...",fear,32,Woman,white
4,img11_reflection.jpg,"[0.7846647500991821, 0.7575387954711914, 1.717...",neutral,36,Woman,white


In [28]:
def read_config(database_name):
    config = configparser.ConfigParser()
    config.read('../config.ini')
    #config.read('config.ini')
    return config[database_name]

In [29]:
config_data = read_config('database')

In [31]:
host = config_data['server']
database = config_data['database']
username = config_data['username']
password = config_data['password']

In [32]:
host

'localhost'

In [36]:
db_connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

cursor = db_connection.cursor()

for index, instance in tqdm(df.iterrows(), total=df.shape[0]):
    img_name = instance["img_name"]
    embeddings = instance["embedding"]
    age = instance["age"]
    dominant_emotion = instance["dominant_emotion"]
    dominant_gender = instance["dominant_gender"]
    dominant_race = instance["dominant_race"]
     
    # Convert embeddings to JSON string
    embeddings_json = json.dumps(embeddings)
    
    # Insert into face_meta table
    current_time = datetime.now()
    unique_key = hashlib.sha256(str(current_time).encode()).hexdigest()

    # Insert into face_meta table
    #insert_statement = "INSERT INTO face_meta (ID, IMG_NAME, EMBEDDING, AGE, DOMINANT_EMOTION, DOMINANT_GENDER, DOMINANT_RACE) VALUES (?, ?, ?, ?, ?, ?, ?)"
    insert_statement = "INSERT INTO face_meta (ID, IMG_NAME, EMBEDDING, AGE, DOMINANT_EMOTION, DOMINANT_GENDER, DOMINANT_RACE) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    insert_args = (unique_key, img_name, embeddings_json, age, dominant_emotion, dominant_gender, dominant_race)
    #print(insert_args)
    cursor.execute(insert_statement, insert_args)
   
    # Insert into face_embeddings table
    for i, embedding in enumerate(embeddings):
        insert_statement = "INSERT INTO face_embeddings (FACE_ID, DIMENSION, VALUE) VALUES (%s, %s, %s)"
        insert_args = (unique_key, i, embedding)
        cursor.execute(insert_statement, insert_args)



100%|██████████| 57/57 [00:02<00:00, 20.64it/s]


In [37]:
db_connection.commit()

In [38]:
cursor.close()
db_connection.close()